`weave` objects controls the output behaviours of a string containing source code.

Converting raw `html` to `vdom`.

In [1]:
import html, vdom.svg, functools, inspect

`dom` are a collection or existing `vdom` tags; if we can't find a tag then we make our won `vdom.createComponent`.

In [2]:
dom = {**vars(vdom), **vars(vdom.svg)}

In [40]:
    def style_dict(object)->dict: 
        """Convert a string of style to a dictionary."""
        return  dict(map(str.strip, object.split(':', 1)) for object in object.split(';') if object) if str else {}

In [71]:
from html.parser import HTMLParser

class html_to_vdom(HTMLParser):
    buf = None    
    def flush(self):
        while len(self.buf) > 1: 
            a = self.buf.pop()
            
            self.append(a)
        return self.buf.pop()
    
    def feed(self, str, tag=vdom.span):
        self.buf = self.buf or [tag()]
        str and super().feed(str)
    
    def __call__(self, str, tag=vdom.div):
        self.buf = [tag()]
        return self.feed(str) or self.flush() or self.buf[0]
    
    def handle_starttag(self, tag, attrs): 
        tag = dom.get(tag, vdom.create_component(tag))
        attrs = dict(attrs)
        self.buf.append(tag(**attrs, style=style_dict(attrs.pop('style', ''))))
        
    def append(self, object):
        self.feed('')
        tag = self.buf[-1].tag_name
        self.buf[-1] = dom.get(tag, vdom.create_component(tag))(
            *self.buf[-1].children, object, style=self.buf[-1].style, attributes=self.buf[-1].attributes
        )

    def handle_endtag(self, tag):
        self.append(self.buf.pop() if self.buf and self.buf[-1].tag_name == tag else F"</{tag}>")
    
    handle_data = append

In [72]:
    def test_graphviz():
        assert html_to_vdom()(__import__('graphviz').Source('graph {A -- B}')._repr_svg_())